In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
import pydotplus
from sklearn.datasets import load_iris
from sklearn import tree
import collections

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

In [2]:
# team data processing
def getTeamStats(year):
    indices = ["", " (1)", " (2)", " (3)", " (4)"]
    o = []
    for i in indices:
        txt = "teamstats/" + str(year) + "/offense/team-stats" + i + ".csv"
        o.append(pd.read_csv(txt))
    tot = o[0]
    tot['curYear'] = year
    tot['nextYear'] = year + 1
    for i in range(1, len(o)):
        tot = pd.merge(tot, o[i], on="TeamName")
        
    d = []
    for i in indices:
        txt = "teamstats/" + str(year) + "/defense/team-stats" + i + ".csv"
        d.append(pd.read_csv(txt))
    for i in range(0, len(d)):
        tot = pd.merge(tot, d[i], on="TeamName")
    
        
        
#     o1 = "data/teamstats/" + str(year) + "offense/team-stats.csv"
#     o2 = "data/teamstats/" + str(year) + "offense/team-stats (1).csv"
#     o3 = "data/teamstats/" + str(year) + "offense/team-stats (2).csv"
#     o4 = "data/teamstats/" + str(year) + "offense/team-stats (3).csv"
#     o5 = "data/teamstats/" + str(year) + "offense/team-stats (4).csv"
#     offense = pd.merge(o1,o2, on="TeamName")
#     offense = pd.merge(offense,o3, on="TeamName")
#     offense = pd.merge(offense,o4, on="TeamName")
#     offense = pd.merge(offense,o5, on="TeamName")
    
#     d1 = "data/teamstats/" + str(year) + "defense/team-stats.csv"
#     d2 = "data/teamstats/" + str(year) + "defense/team-stats (1).csv"
#     d3 = "data/teamstats/" + str(year) + "defense/team-stats (2).csv"
#     d4 = "data/teamstats/" + str(year) + "defense/team-stats (3).csv"
#     d5 = "data/teamstats/" + str(year) + "defense/team-stats (4).csv"
#     defense = pd.merge(o1,o2, on="TeamName")
#     defense = pd.merge(defense,o3, on="TeamName")
#     defense = pd.merge(defense,o4, on="TeamName")
#     defense = pd.merge(defense,o5, on="TeamName")
#     total = pd.merge(offense, defense, on="Teamname")
    return tot
    
l = []
for i in range(2010, 2018):
    x = getTeamStats(i)
    l.append(x)
d = pd.concat(l)
d = d.reset_index()

d.to_csv("data.csv")


In [3]:
# d.TimeOfPossession[0].split(":")

def getTimeofP(row):
    x, y = row.TimeOfPossession_x.split(":")
    m = int(x)
    n = int(y)
    return m*60 + n
d["TimeOfPossession"] = d.apply(getTimeofP, axis=1)

In [4]:
# get playoff teams
playoffs = {}
playoffs[2010] = ["Patriots", "Steelers", "Falcons", "Bears", "Jets", "Colts" "Ravens", "Chiefs", "Packers", "Eagles", "Saints", "Seahawks"]
playoffs[2011] = ["Patriots", "Ravens", "Packers", "49ers", "Falcons", "Giants", "Lions", "Bengals", "Texans", "Steelers", "Broncos", "Saints"]
playoffs[2012] = ["Broncos", "Patriots", "Texans", "Ravens", "Colts", "Bengals", "Ffalcons", "49ers", "Packers", "Redskins", "Seahawks", "Vikings"]
playoffs[2013] = ["Broncos", "Patriots", "Bengals", "Colts", "Chiefs", "Chargers", "Seahawks", "Panthers", "Eagles", "Packers", "49ers", "Saints"]
playoffs[2014] = ["Patriots", "Broncos", "Steelers", "Colts", "Bengals", "Ravens", "Seahawks", "Packers", "Cowboys", "Panthers", "Cardinals", "Lions"]
playoffs[2015] = ["Broncos", "Patriots", "Bengals", "Texans", "Chiefs", "Steelers", "Panthers", "Cardinals", "Vikings", "Redskins", "Packers", "Seahawks"]
playoffs[2016] = ["Patriots", "Chiefs", "Steelers", "Texans", "Raiders", "Dolphins", "Cowboys", "Falcons", "Seahawks", "Packers", "Giants", "Lions"]
playoffs[2017] = ["Patriots", "Steelers", "Jaguars", "Chiefs", "Titans", "Bills", "Eagles", "Vikings", "Rams", "Saints", "Panters", "Falcons"]

In [6]:
def getPlayoffs(row):
    if row['TeamName'] in playoffs[row["curYear"]]:
        return 1
    else:
        return 0
    
#     curYear = 
#     df['p'] = 0
#     for index, row in df.iterrows():
#         if df.loc[index, "TeamName"] in playoffs[int(df.loc[index, 'curYear'])]:
#             df.loc[index, 'p'] = 1
#     return df

In [42]:
def getBest(df):
    d = {}
    for index, row in df.iterrows():
        if row.TeamName not in d:
            d[row.TeamName] = 0
        d[row.TeamName] +=1
    return d


In [7]:
d['Playoffs'] = d.apply(getPlayoffs, axis = 1)
d = d.ix[:,~d.columns.duplicated()]

In [48]:
# df['Auction_Rank'] = df.groupby('Auction_ID')['Bid_Price'].rank(ascending=False)
d['PointsRank'] = d.groupby("curYear")['PointsPerGame_x'].rank(ascending=False)
d['TurnoverRank'] = d.groupby("curYear")['TurnoverDifferential'].rank(ascending=False)
d['OppOffenseRank'] = d.groupby("curYear")['OpponentOffensiveYards'].rank(ascending=True)
d['TOPrank'] = d.groupby("curYear")['TimeOfPossession'].rank(ascending=False)
d['RushRank'] = d.groupby("curYear")['RushingYards_x'].rank(ascending=False)
d_mostpoints = d.loc[d['PointsRank']<=3]
d_turnrank = d.loc[d['TurnoverRank']<=3]
d_oppoffrank = d.loc[d['OppOffenseRank']<=3]
d_top = d.loc[d['TOPrank']<=3]
d_rushrank = d.loc[d['RushRank']<=3]

In [49]:
d_mostpoints.loc[:,['TeamName', 'curYear']]
x = getBest(d_mostpoints)
x

{'Bears': 1,
 'Broncos': 3,
 'Cardinals': 1,
 'Chargers': 1,
 'Eagles': 3,
 'Falcons': 1,
 'Packers': 2,
 'Panthers': 1,
 'Patriots': 7,
 'Rams': 1,
 'Saints': 3}

In [61]:
#chiefs?
x = getBest(d_turnrank)
x
d_turnrank.loc[:,['TeamName', 'curYear', "TurnoverRank"]]

,TeamName,curYear,TurnoverRank
7,Patriots,2010,1.0
13,Steelers,2010,2.0
15,Falcons,2010,3.0
33,Patriots,2011,3.0
34,Packers,2011,2.0
57,49ers,2011,1.0
64,Patriots,2012,1.0
68,Redskins,2012,3.0
91,Bears,2012,2.0
110,Colts,2013,3.0


In [62]:
#broncos
x = getBest(d_oppoffrank)
x
d_oppoffrank.loc[:,['TeamName', 'curYear', "OppOffenseRank"]]

,TeamName,curYear,OppOffenseRank
0,Chargers,2010,1.0
10,Jets,2010,3.0
13,Steelers,2010,2.0
43,Steelers,2011,1.0
44,Texans,2011,2.0
46,Ravens,2011,3.0
67,Broncos,2012,2.0
74,49ers,2012,3.0
84,Steelers,2012,1.0
105,Bengals,2013,3.0


In [53]:
x = getBest(d_top)
x
d_top.loc[:,['TeamName', 'curYear', 'TOPrank']]

,TeamName,curYear,TOPrank
26,Titans,2010,1.0
30,Cardinals,2010,2.0
31,Panthers,2010,3.0
59,Seahawks,2011,3.0
61,Colts,2011,1.0
62,Rams,2011,2.0
88,Browns,2012,2.0
89,Titans,2012,3.0
92,Jaguars,2012,1.0
97,Eagles,2013,1.0


In [60]:
x = getBest(d_rushrank)
x
d_rushrank.loc[:,['TeamName', 'curYear', 'RushRank']]

,TeamName,curYear,RushRank
9,Raiders,2010,2.0
11,Chiefs,2010,1.0
14,Jaguars,2010,3.0
38,Panthers,2011,3.0
44,Texans,2011,2.0
54,Broncos,2011,1.0
68,Redskins,2012,1.0
80,Seahawks,2012,3.0
83,Vikings,2012,2.0
97,Eagles,2013,1.0


In [54]:
# features will be 
# data_train = d.loc[:,["TeamName", "PointsPerGame_x"]]
features = ["PointsPerGame_x" , 'ThirdDownPercentage', 'RedZonePercentage', 'OffensiveYardsPerPlay', 'PenaltyYards', 'TurnoverDifferential', 'RushingYards_x', 'TimesSacked_x', "OpponentTimesSacked_x", 'PassingYards_x', 'OpponentRedZonePercentage'
            ,'OpponentOffensiveYards'
#             , 'OpponentPassingYards_x', 'OpponentRushingYards_x',
#             'PasserRating','TimeOfPossession',
           ]
data_train_x = d.loc[:,features]
data_train_y = d.loc[:, ["Playoffs"]]

In [55]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(data_train_x, data_train_y)

In [56]:
dot_data = tree.export_graphviz(clf,
                                feature_names=features,
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')

True

In [11]:
np.any(np.isnan(data_train_x))
# np.all(np.isfinite(data_train_x))

False

In [12]:
data_train_x.shape

(256, 15)

In [13]:
data_train_y.shape

(256, 1)

In [161]:
data_train_x.loc[data_train_x.isnull().any(axis=1) == 1]

,PointsPerGame_x,ThirdDownPercentage,RedZonePercentage,OffensiveYardsPerPlay,PenaltyYards,TimeOfPossession,TurnoverDifferential,RushingYards_x,TimesSacked_x,PassingYards_x,OpponentRedZonePercentage,OpponentPassingYards_x,OpponentRushingYards_x,PasserRating
